<a href="https://colab.research.google.com/github/kiks10/reasoning-ass1/blob/main/ReasoningAssign1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Eliminate implication

In [ ]:
import re


def eliminate_implication(sentence):
    while '=>' in sentence:
        pos = sentence.find('=>')
        left = sentence[:pos].strip()
        right = sentence[pos + 2:].strip()
        sentence = '(' + '~' + left + ') or (' + right + ')'
    return sentence

def apply_demorgan(clause):
    if 'and' in clause:
        parts = clause.split('and')
        clause = '(~' + parts[0].strip() + ' or ~' + parts[1].strip() + ')'

    elif 'or' in clause:
        parts = clause.split('or')
        clause = '(~' + parts[0].strip() + ' and ~' + parts[1].strip() + ')'

    return clause

def get_inner_part(index, clause):
    stack = ['(']
    new_clause = ""
    while len(stack) != 0:
        if clause[index] == '(':
            stack.append('(')
        elif clause[index] == ')':
            stack.pop()
        new_clause += clause[index]
        index += 1
    return new_clause, index

def move_negation_inward(sentence):
    if "~(" in sentence:
        negation_index = sentence.index("~(")
        inner_part, closing_index = get_inner_part(negation_index + 2, sentence)
        inner_part = apply_demorgan(inner_part)
        inner_part = move_negation_inward(inner_part)
        return sentence[:negation_index] + inner_part + sentence[closing_index + 1:]

    if "~forall" in sentence:
        sentence = sentence.replace("~forall", "exist ~")
    elif "~exist" in sentence:
        sentence = sentence.replace("~exist", "forall ~")

    return sentence

def remove_double_negations(sentence):
    while "~~" in sentence:
        sentence = sentence.replace("~~", "")
    return sentence

def standardize_variables(sentence):
    variables_list = []

    quantifiers = re.findall(r'[∀∃]', sentence)

    for quantifier in quantifiers:
        quantifier_index = sentence.index(quantifier)

        variable = sentence[quantifier_index + 1]

        if variable in variables_list:
            i = 1
            while True:
                new_variable = chr(ord(variable) + i)
                if new_variable not in variables_list:
                    break
                i += 1
            sentence = sentence[:quantifier_index + 1] + new_variable + sentence[quantifier_index + 2:]

            variables_list.append(new_variable)
        else:
            variables_list.append(variable)

    for i in range(len(sentence)):
        if sentence[i] == '(':
            start_index = i
        elif sentence[i] == ')':
            end_index = i
            for j in range(start_index + 1, end_index):
                if sentence[j].isalpha():
                    sentence = sentence[:j] + sentence[j-4] + sentence[j+1:]

    return sentence


def prenex_form(sentence):
    quantifiers = []
    while 'exists' in sentence or 'forall' in sentence:
        for i, char in enumerate(sentence):
            if char in ['e', 'f']:
                if sentence[i:i+6] == 'exists':
                    quantifiers.append('exists')
                elif sentence[i:i+6] == 'forall':
                    quantifiers.append('forall')
                break
        sentence = sentence.replace(quantifiers[-1], '', 1)
    return ' '.join(quantifiers) + sentence



def skolemize(sentence):
    universal_pattern = r'forall\s*([a-zA-Z]+)\s'

    existential_pattern = r'exists\s*([a-zA-Z]+)\s*'

    #findind all forall & exists in sentence
    universal_matches = re.findall(universal_pattern, sentence)
    existential_matches = re.findall(existential_pattern, sentence)

    skolemized_expression = sentence

    #if there is forall before exists
    if not universal_matches:
        for variable in existential_matches:
            skolemized_expression = skolemized_expression.replace(f'exists {variable}', '')
            skolemized_expression = skolemized_expression.replace(f'{variable}', f'F(x)')
    else:
    #if there isn't forall before exists
        for char in existential_matches:
            skolemized_expression = skolemized_expression.replace(f'exists {char}', '')
            skolemized_expression = re.sub(f'{char}', f'F({universal_matches[0]})', skolemized_expression)

    return skolemized_expression



def eliminate_universal_quantifiers(sentence):
    return sentence.replace('forall ', '')

def convert_to_cnf(clauses) :
    transformed_clauses = []
    for clause in clauses:
        clause = eliminate_implication(clause)
        clause = move_negation_inward(clause)
        clause = remove_double_negations(clause)
        clause = standardize_variables(clause)
        clause = prenex_form(clause)
        clause =skolemize(clause)
        clause =eliminate_universal_quantifiers(clause)
        transformed_clauses.append(clause)

    return transformed_clauses

def turn_conjunctions_into_clauses(sentence):
    clauses = sentence.split('and')
    return set(clauses)



def rename_variables_in_clauses(clauses):
    renamed_clauses = []
    for clause in clauses:
        variables = re.findall(r'\b[a-zA-Z]+\b', clause)  # Find all variable names using regex
        replacements = {}
        for var in variables:
            new_var = var + '1'
            replacements[var] = new_var
            clause = re.sub(r'\b' + re.escape(var) + r'\b', new_var, clause)  # Replace variable names
        renamed_clauses.append(clause)
    return renamed_clauses



def cancel_complementary_literals(clauses):

    for i, clause in enumerate(clauses):
        literals = clause.split(',')
        updated_literals = []


        for literal in literals:
            complement = literal.strip('~') if '~' in literal else '~' + literal
            if complement in literals:

                literals.remove(complement)
                literals.remove(literal)
            else:

                updated_literals.append(literal)


        clauses[i] = ','.join(updated_literals)

    return clauses

def check_consistency(clauses):
    while True:

        updated_clauses = cancel_complementary_literals(clauses)

        if all(not clause for clause in updated_clauses):
            return True

        if updated_clauses == clauses:
            return False

        clauses = updated_clauses




def main():


    expression = "~(x and y)"
    sentence = move_negation_inward(expression)
    print("After moving negation inward:", sentence)

    expression = "(∀k P(k)) ∨ (∃k Q(k))"
    sentence = standardize_variables(expression)
    print("After standardizing variable scope:", sentence)

    sentence = ' forall x (P(x) => exists y (Q(y) and ~~ R(z)))'
    print("Original sentence:", sentence)

    sentence = eliminate_implication(sentence)
    print("After eliminating implication:", sentence)

    sentence = skolemize(sentence)
    print("After skolimizing:", sentence)

    sentence = eliminate_universal_quantifiers(sentence)
    print("After eliminating universal quantifiers:", sentence)

    sentence = prenex_form(sentence)
    print("After converting to prenex form:", sentence)

    sentence = remove_double_negations(sentence)
    print("After removing double not:", sentence)

    clauses = turn_conjunctions_into_clauses(sentence)
    print("Clauses:", clauses)


    renamed_test_clauses = rename_variables_in_clauses(clauses)
    print("Renamed clauses:", renamed_test_clauses)


    test_clauses = ["~t", "~p", "~p or t"]
    consistent = check_consistency(test_clauses)
    if consistent:
     print("The clauses are consistent.")
    else:
     print("The clauses are inconsistent.")


















if __name__ == "__main__":
    main()


After moving negation inward: (~x or ~y))
After standardizing variable scope: (∀k P(k)) ∨ (∃l Q(l))
Original sentence:  forall x (P(x) => exists y (Q(y) and ~~ R(z)))
After eliminating implication: (~forall x (P(x)) or (exists y (Q(y) and ~~ R(z))))
After skolimizing: (~forall x (P(x)) or ( (Q(F(x)) and ~~ R(z))))
After eliminating universal quantifiers: (~x (P(x)) or ( (Q(F(x)) and ~~ R(z))))
After converting to prenex form: (~x (P(x)) or ( (Q(F(x)) and ~~ R(z))))
After removing double not: (~x (P(x)) or ( (Q(F(x)) and  R(z))))
Clauses: {'(~x (P(x)) or ( (Q(F(x)) ', '  R(z))))'}
Renamed clauses: ['(~x1 (P1(x1)) or1 ( (Q1(F1(x1)) ', '  R1(z1))))']
The clauses are inconsistent.
